In [1]:
# Установка
%pip install langchain langchain-community gigachat
%pip install --upgrade langchain-gigachat


In [2]:
# Настройка
import os
from getpass import getpass
from langchain_gigachat.chat_models import GigaChat

In [3]:
# Получаем токен (или логин/пароль)
if "GIGACHAT_CREDENTIALS" not in os.environ:
    os.environ["GIGACHAT_CREDENTIALS"] = getpass("Введите ваши credentials GigaChat: ")

KeyboardInterrupt: Interrupted by user

In [4]:
# Инициализация LLM
from langchain_community.chat_models import GigaChat
llm = GigaChat(verify_ssl_certs=False, scope="GIGACHAT_API_PERS")

<ipython-input-4-6f477a5e6911>:3: LangChainDeprecationWarning: The class `GigaChat` was deprecated in LangChain 0.3.5 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-gigachat package and should be used instead. To use it run `pip install -U :class:`~langchain-gigachat` and import as `from :class:`~langchain_gigachat import GigaChat``.
  llm = GigaChat(verify_ssl_certs=False, scope="GIGACHAT_API_PERS")


In [5]:
# Установка (один раз в терминале)
# pip install --upgrade langchain langchain-gigachat

from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA

# Новый импорт GigaChat из отдельного пакета
from langchain_gigachat.chat_models import GigaChat

In [ ]:
!pip install unstructured
!pip install "unstructured[pdf]"

In [7]:
!mkdir -p /content/docs
!cp /content/sample_document.pdf /content/docs/


In [8]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyPDFLoader

# Загружаем все PDF-файлы из папки /content/docs/
loader = DirectoryLoader("./docs", glob="*.pdf", loader_cls=PyPDFLoader)
docs = loader.load()
print(f"Загружено документов: {len(docs)}")


Загружено документов: 47


In [9]:
# 2. Разбиение на чанки
splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
chunks = splitter.split_documents(docs)

In [10]:
%pip install faiss-gpu-cu12



  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [11]:
print(f"Loaded documents: {len(docs)}")
for d in docs:
    print(f" - {d.metadata.get('source', '')}")

Loaded documents: 47
 - docs/sample_document.pdf
 - docs/sample_document.pdf
 - docs/sample_document.pdf
 - docs/sample_document.pdf
 - docs/sample_document.pdf
 - docs/sample_document.pdf
 - docs/sample_document.pdf
 - docs/sample_document.pdf
 - docs/sample_document.pdf
 - docs/sample_document.pdf
 - docs/sample_document.pdf
 - docs/sample_document.pdf
 - docs/sample_document.pdf
 - docs/sample_document.pdf
 - docs/sample_document.pdf
 - docs/sample_document.pdf
 - docs/sample_document.pdf
 - docs/sample_document.pdf
 - docs/sample_document.pdf
 - docs/sample_document.pdf
 - docs/sample_document.pdf
 - docs/sample_document.pdf
 - docs/sample_document.pdf
 - docs/sample_document.pdf
 - docs/sample_document.pdf
 - docs/sample_document.pdf
 - docs/sample_document.pdf
 - docs/sample_document.pdf
 - docs/sample_document.pdf
 - docs/sample_document.pdf
 - docs/sample_document.pdf
 - docs/sample_document.pdf
 - docs/sample_document.pdf
 - docs/sample_document.pdf
 - docs/sample_document.pdf

In [12]:
import numpy as np
print(type(chunks[0]))



<class 'langchain_core.documents.base.Document'>


In [13]:
!pip install numpy==1.26.4


In [14]:
!pip install --force-reinstall transformers sentence-transformers

  Using cached transformers-4.51.3-py3-none-any.whl.metadata (38 kB)
  Using cached sentence_transformers-4.1.0-py3-none-any.whl.metadata (13 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached huggingface_hub-0.30.2-py3-none-any.whl.metadata (13 kB)
  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached regex-2024.11.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached safetensors-0.5.3-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (

In [15]:
!pip install -U langchain-huggingface
!pip install sentence_transformers transformers

In [16]:
from langchain_community.embeddings import HuggingFaceEmbeddings
import numpy as np
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

# 1. Инициализируем эмбеддинги
emb = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    model_kwargs={"device": "cpu"},
    encode_kwargs={"convert_to_numpy": True}
)

# 2. Извлекаем тексты
texts = [doc.page_content for doc in chunks]

# 3. Получаем эмбеддинги
raw_embeds = emb.embed_documents(texts)
embeds_np = np.array(raw_embeds, dtype=np.float32)

# 4. FAISS index
dim = embeds_np.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeds_np)

# 5. Векторное хранилище
vectordb = FAISS(
    embedding_function=emb,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={i: str(i) for i in range(len(chunks))},
)

# 6. Ретривер
retriever = vectordb.as_retriever(search_kwargs={"k": 5})


<ipython-input-16-643bb35f9844>:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  emb = HuggingFaceEmbeddings(


RuntimeError: Failed to import transformers.models.auto.modeling_auto because of the following error (look up to see its traceback):
Failed to import transformers.generation.utils because of the following error (look up to see its traceback):
module 'numpy' has no attribute '_no_nep50_warning'

In [ ]:
# 4. Инициализация LLM (GigaChat)
llm = GigaChat(
    credentials="Y2M2NWE4NmItN2M4ZS00MDY4LTg5YzQtOWZiN2NlYjZlODdjOjc1NWM4MmM5LTczM2MtNDIwOC05ZDQzLWYxYWQyZDM5ZjlhNQ==",
    scope="GIGACHAT_API_PERS",
    verify_ssl_certs=False,
)

In [ ]:
# 5. Создание RAG‑цепочки и генерация ответа
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
)

In [ ]:
answer = qa.run("")
print(answer)


--------

In [1]:
# Установка
%pip install langchain langchain-community gigachat

# Настройка
import os
from getpass import getpass

# Получаем токен (или логин/пароль)
if "GIGACHAT_CREDENTIALS" not in os.environ:
    os.environ["GIGACHAT_CREDENTIALS"] = getpass("Введите ваши credentials GigaChat: ")

# Инициализация LLM
from langchain_community.chat_models import GigaChat
llm = GigaChat(verify_ssl_certs=False, scope="GIGACHAT_API_PERS")


Введите ваши credentials GigaChat: ··········


<ipython-input-1-40c884313531>:14: LangChainDeprecationWarning: The class `GigaChat` was deprecated in LangChain 0.3.5 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-gigachat package and should be used instead. To use it run `pip install -U :class:`~langchain-gigachat` and import as `from :class:`~langchain_gigachat import GigaChat``.
  llm = GigaChat(verify_ssl_certs=False, scope="GIGACHAT_API_PERS")


In [2]:
%pip install unstructured
%pip install "unstructured[pdf]"

In [10]:
%pip install --upgrade langchain-huggingface

In [3]:
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA


In [4]:
# 1. Ingestion
loader = DirectoryLoader("docs/", glob="**/*.pdf")
docs = loader.load()


In [5]:
# 2. Splitting
splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=200)
chunks = splitter.split_documents(docs)

In [16]:
%pip install numpy

In [17]:
%pip install faiss-gpu-cu11


  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 MB 23.9 MB/s eta 0:00:00
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 38.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [6]:
# 3. Эмбеддинги
emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
vectordb = FAISS.from_documents(chunks, emb)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
# 4. Retriever
retriever = vectordb.as_retriever(search_kwargs={"k":5})

In [8]:
# 5. LLM — пример с GigaChat
from langchain_community.chat_models import GigaChat
llm = GigaChat(verify_ssl_certs=False, scope="GIGACHAT_API_PERS")

In [9]:

# 6. Цепочка RetrievalQA
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever
)

In [10]:
answer = qa.run("Что такое RAG‑система?")
print(answer)


<ipython-input-10-33b50bba2c0a>:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa.run("Что такое RAG‑система?")


Из предоставленного контекста не удается однозначно определить, что такое RAG-система. Возможно, это сокращение от названия компании или метода, но информация недостаточна для точного ответа.
